# The Melting Pot of Berlin

### Introduction

Have you ever asked yourself where currywurst was invented? The invention of currywurst is attributed to a woman called Herta Heuwer in Berlin in 1949, after she obtained ketchup and curry powder from British soldiers in Germany. She mixed these ingredients with other spices and poured it over grilled pork sausage. Heuwer started selling the cheap but filling snack at a street stand in the Charlottenburg district of Berlin, where it became popular with construction workers rebuilding the devastated city. Although, currywurst is the most popular and well-known food of Berlin, the city still can offer a lot more. Especially when it comes to international food, since it's the second home for more than million people with migration background. Therfore, it's somehow interesting to learn more about food in Berlin and how migration affects local food and traditional restaurants. 
In the following notebook, we will analyze the impact of migration on local and traditional restaurants in Berlin based on the distribution of local restaurants around the city.

### Data

Therefor, we will use official data from the Statistical Office of Berlin-Brandenburg about registered residents with migration background which was published by the Federal State of Berlin in 2018. Moreover, we will be using data about the boroughs of Berlin, also published by the Federal State, in combination with Foursquare location data in order to learn more about local restaurants in Berlin. So let's get started!

### Code

Download all dependencies that we will need.

In [ ]:
import numpy as np # Library to handle data in a vectorized manner

import pandas as pd # Library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # Library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # Library to handle requests
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

import bs4 as bs
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 

Download data about registered residents with migration background from csv-file.

In [ ]:
br_data = pd.read_csv("https://www.statistik-berlin-brandenburg.de/opendata/EWRMIGRA201812H_Matrix.csv", sep=";")
br_data.head()

Let's take a look at the structure of the dataframe.

In [ ]:
print("The dataframe has {} rows and {} columns.".format(br_data.shape[0], br_data.shape[1]))

Let's take a look at the data types of the columns.

In [ ]:
br_data.dtypes

As you can see all columns contain data of type int.

Let's get a statistical summary of the data.

In [ ]:
br_data.describe()

As you can see for some columns it's not that useful to determine descriptive statistics. So let's start cleaning the data by dropping all columns that we won't use in our analysis.

In [ ]:
br_data.drop(["ZEIT","RAUMID", "PGR", "BZR", "PLR", "STADTRAUM", "HK_EU15", "HK_EU28", "HK_Polen", "HK_EheJug", "HK_EheSU", "HK_Turk","HK_Arab", "HK_Sonst", "HK_NZOrd"], axis=1, inplace=True)

# BEZ stands for the Admin. Nr. of the borough and MH_E for the total number of registered people with migration background. So let's rename them.
br_data.rename(columns={"BEZ": "Ad. Nr.", "MH_E":"People with migration background"}, inplace=True)

# Sort values by Ad. Nr. in ascending order 
br_data.sort_values("Ad. Nr.", inplace=True)

# Group data by Ad. Nr. and count the sum of people with migration background in each borough
br_data = br_data.groupby("Ad. Nr.").sum()
br_data

Now, we need the official names of the boroughs. So let's download them by importing the following csv-file.

In [ ]:
brgh = pd.read_csv("https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.csv")
brgh.head()

As you can see, the dataframe needs some cleaning work.

Therefor, rename important columns, sort the data by Ad. Nr. in ascending order and set it as index.

In [ ]:
brgh.rename(columns={"Gemeinde_schluessel": "Ad. Nr.", "Gemeinde_name":"Borough"}, inplace=True)
brgh.sort_values("Ad. Nr.", ascending=True, inplace=True)
brgh.reset_index(drop=True, inplace=True)
brgh.set_index("Ad. Nr.", inplace=True)
brgh

Now, we will merge both dataframes and set Ad. Nr. as index.

In [ ]:
brgh = brgh[["Borough"]]
brgh["People with migration background"] = br_data["People with migration background"]
br_data = brgh
br_data

Now, let's get geolocation data for each borough.

In [ ]:
# Import dependencies
from geopy.extra.rate_limiter import RateLimiter 

locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Berlin, DE")

# Conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# Create location column
br_data['Location'] = br_data['Borough'].apply(geocode)

# Create longitude, laatitude and altitude from location column (returns tuple)
br_data['Point'] = br_data['Location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Split point column into latitude, longitude and altitude columns
br_data[['Latitude', 'Longitude', 'Altitude']] = pd.DataFrame(br_data['Point'].tolist(), index=br_data.index)
br_data.head()

Let's drop unnecessary columns like Location, Point and Altitude.

In [ ]:
br_data.drop(["Location", "Point", "Altitude"], axis=1, inplace=True)
br_data

As you may have noticed, the latitude and longitude of Lichtenberg are obviously wrong. So let's correct them.

In [ ]:
location = "Lichtenberg, Berlin, DE"

geolocator = Nominatim(user_agent = "br_explorer")
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinates of Lichtenberg are {}, {}.".format(latitude, longitude))

Now, we have the right coordinates. Let's add them to our data.

In [ ]:
br_data.loc[11,"Latitude"] = latitude
br_data.loc[11,"Longitude"] = longitude
br_data

And there you have it, a nice dataframe with all the data we need.

In the **second step**, we will use Foursquare location data to learn more about local restaurants in Berlin.

Let's define Foursquare credentials and version.

In [ ]:
CLIENT_ID = "HQHI11JH4DAVEGSIYZHDM4BA5DFETP3MQBXKJOSV4X3YX4RN" # Foursquare ID
CLIENT_SECRET = "ITTCVGBQRSJVDQQDGPCJ3SDUG5FV5B1MHBGYRAYQJLH2DT0V" # Foursquare Secret
VERSION = "20180605" # Foursquare API version

print("Your credentails:")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET:" + CLIENT_SECRET)

Let's explore the boroughs of Berlin. We can start with Berlin Mitte, since it's one of the largest boroughs in Berlin.

In [ ]:
br_loc = br_data.loc[1, "Borough"] # borough name
br_lat = br_data.loc[1, "Latitude"] # neighborhood latitude value
br_long = br_data.loc[1, "Longitude"] # neighborhood longitude value

print("Latitude and longitude values of {} are {}, {}.".format(br_loc, br_lat, br_long))

Now, let's get the top 100 venues which are located in **Berlin Mitte**.

In [ ]:
radius = 5000 # Search radius in m
search_query = 'venues' # Search query
LIMIT = 100 # Results limit
exp_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,
                                                                                                                               br_lat, br_long,
                                                                                                                               VERSION, radius, LIMIT)
exp_url

Send the GET request and examine the results.

In [ ]:
results = requests.get(exp_url).json()

Now, extract the category of the venues above in order to get restaurants data and clean the results.

In [ ]:
def get_category_type(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]["name"]
    
venues = results["response"]["groups"][0]["items"]

# flatten JSON
nearby_venues = json_normalize(venues)
# filter columns
filtered_columns = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

Now, we have a dataframe with all venues within a radius of 5000 m around **Berlin Mitte**. Let's take a look at it.

In [ ]:
nearby_venues.head()

Now, let's create a function to apply the same process on all boroughs.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Borough", 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each borough, create a new dataframe and name it **br_venues**.

In [ ]:
br_venues = getNearbyVenues(names=br_data["Borough"],
                                   latitudes=br_data["Latitude"],
                                   longitudes=br_data["Longitude"]
                                  )

Let's check the size of the resulting dataframe and take a look at it.

In [ ]:
print(br_venues.shape)
br_venues.head()

Now, let's filter our venues in order to get an extra dataframe for restaurants.

In [ ]:
idx = np.where((br_venues['Venue Category'].str.endswith('Restaurant')))
br_rest = br_venues.loc[idx]
br_rest.reset_index(drop=True, inplace=True)
br_rest.drop("Borough Latitude",axis=1, inplace=True)
br_rest.drop("Borough Longitude",axis=1, inplace=True)
br_rest.rename(columns={"Venue": "Restaurant", "Venue Latitude":"Latitude", "Venue Longitude":"Longitude", "Venue Category":"Category"}, inplace=True)
print(br_rest.shape)
br_rest.head()

Let's check how many restaurants were returned for each category.

In [ ]:
br_rest.groupby(["Category"]).count().head(15)

Let's find out how many unique categories can be curated from all the returned restaurants.

In [ ]:
print("There are {} unique categories.".format(len(br_rest["Category"].unique())))

In order to analyze local restaurants, let's filter the category "German Restaurant".

In [ ]:
idx = np.where((br_rest['Category'].str.startswith('German'))) # Filter category "German"
br_rest_gr = br_rest.loc[idx]
br_rest_gr = br_rest_gr.groupby("Borough").count() # Group by bourough name and count
br_rest_gr.reset_index(inplace=True) # Reset index
br_rest_gr.head()

Now, let's try to visulize both dataframes br_data and  br_rest_gr using folium and choropleth maps. Therefor, download geojson file for boroughs.

In [ ]:
# download boroughs geojson file
!wget --quiet https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.geojson
print("Download completed.")

Create a map of Berlin using folium.

In [ ]:
berlin_geo = r'https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.geojson' # Read geojson file
lat = 52.5
lon = 13.42

# Create map
berlin_map1 = folium.Map(location=[lat, lon], zoom_start=10)
berlin_map1

Now, generate a choropleth map using the total number of registered people with migration background in Berlin.

In [ ]:
# Generate choropleth map 
berlin_map1.choropleth(
    geo_data=berlin_geo,
    data=br_data,
    columns=['Borough', 'People with migration background'],
    key_on='feature.properties.Gemeinde_name',
    fill_color='Blues', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='People with migration background in Berlin'
)

berlin_map1

Now, let's create a new map of Berlin in order to visualize the distribution of local restaurants in Berlin.

In [ ]:
# Create a new Berlin map
berlin_map2 = folium.Map(location=[lat, lon], zoom_start=10)
berlin_map2

Create choropleth map of German restaurants.

In [ ]:
berlin_map2.choropleth(
    geo_data=berlin_geo,
    data=br_rest_gr,
    columns=['Borough', 'Restaurant'],
    key_on='feature.properties.Gemeinde_name',
    fill_color='Blues', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='German restaurants in Berlin'
)

berlin_map2

As you may have noticed when you compare both choropleth maps, boroughs with a high proportion of people with migration background have generally less local restaurants than boroughs with less proportion of people with migration background.

### Final report

Coming soon